In [1]:
import pandas as pd
from config import BASE_DIR, SEED_INSTANCE_GENERATION_CONFIG

In [2]:
from os.path import join
seed_instances_path = SEED_INSTANCE_GENERATION_CONFIG["seed_instances_output_path"]
reddit_path = join(BASE_DIR, "data/training_data/reddit_data_v1.csv")

In [3]:
reddit_path

'/workspaces/Inner-Critic-Chatbot/data/training_data/reddit_data_v1.csv'

In [4]:
SEED_INSTANCES = pd.read_json(seed_instances_path)
REDDIT_INSTANCES = pd.read_csv(reddit_path, index_col=0)

In [10]:
RANDOM_SEED = 42
REDDIT_SAMPLES = REDDIT_INSTANCES.rename(columns={"post": "input"}).sample(n=250, random_state=RANDOM_SEED)
SEED_SAMPLES = SEED_INSTANCES.sample(n=250, random_state=RANDOM_SEED)

In [11]:
final_df = pd.concat([REDDIT_SAMPLES, SEED_SAMPLES], axis=0).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
final_df

,subreddit,author,date,input,anger,disgust,fear,joy,neutral,sadness,surprise,negative_emotion_weight,dominant_negative_emotion,scenario,theme,emotion,num_sentences,sentence_diversity_level
0,NaN,NaN,NaN,I work so hard to make sure our family is take...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A husband feels underappreciated because his w...,Fear of Failure,Nervous,7.0,High
1,socialanxiety,alpharusin,2020/03/19,Social Anxiety HELP! Hello guys! Can anybody h...,0.700026,0.002039,0.089877,0.087170,0.099963,0.017164,0.003761,0.203265,anger,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,I can't help but feel nervous about my son's e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mother is worried about her son's eating hab...,Fear of Failure,Nervous,3.0,High
3,socialanxiety,bluecanoe_,2020/02/14,Why do people keep looking at me? I try not to...,0.156686,0.271966,0.042593,0.002192,0.479914,0.008616,0.038033,0.120655,anger,NaN,NaN,NaN,NaN,NaN
4,lonely,MonkeyShitStorm,2020/04/04,"I Have No One I have one friend, but he has be...",0.007203,0.024891,0.005088,0.007058,0.182501,0.772486,0.000774,0.203387,sadness,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,lonely,hellsignmeup,2020/03/20,Should he give in? A woman has been trying to ...,0.532791,0.376518,0.003353,0.003858,0.050798,0.017457,0.015224,0.233851,anger,NaN,NaN,NaN,NaN,NaN
496,NaN,NaN,NaN,I can't believe I've been living a lie all the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A married man has recently discovered that he'...,Not being good enough,Angry,4.0,High
497,NaN,NaN,NaN,Why do I always get so worked up before these ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An introverted person feels overwhelmed when h...,Fear of What Others Think,Angry,4.0,Medium
498,NaN,NaN,NaN,I've been feeling really anxious lately. It's ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A girlfriend is struggling with her boyfriend'...,Fear of What Others Think,Nervous,10.0,Medium


In [12]:
combined_data_path = join(BASE_DIR, 'data/training_data/merged_seed_instances_v1.csv')
final_df.to_csv(combined_data_path)

In [16]:
for i, ex in final_df.sample(n=15).iterrows():
    print(f'Example {i} \n:')
    print(ex.input)
    print('\n')

Example 118 
:
"I AM SORRY FOR EXISTING Y'ALL" You ever feel like a burden to your therapist? Like damn, this stuff I'm telling them is so insignificant. I'm wasting their time, &amp; they're probably internally rolling their eyes at me because they deal with people who have real problems.

I apologized for my existence to one of my therapists once. They asked me to imagine myself walking into a room full of people and shouting, "I AM SORRY FOR EXISTING Y'ALL." 

That was funny, and sometimes helps.


Example 499 
:
Just got told that one of my so called “best mates” that i’ve known for 14 years had a birthday gathering last night and I didn’t get invited :))))))) Fucking hell I thought I at least had friends lmao


Example 369 
:
I don't know what's wrong with me. It's like every time I look in the mirror, all I can see are these red, angry bumps on my face. I can't even imagine going to a party or hanging out with my friends because I'm so self-conscious about my acne. It's like I ca